<a href="https://colab.research.google.com/github/demiurge/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir =os.path.join("logs", datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score ):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred) 

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential ([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

In [8]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 20ms/step - loss: 2.7675 - accuracy: 0.2482 - val_loss: 1.3183 - val_accuracy: 0.5932
Epoch 2/5
272/272 [==============================] - 5s 18ms/step - loss: 0.9705 - accuracy: 0.6895 - val_loss: 0.3091 - val_accuracy: 0.8989
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.4321 - accuracy: 0.8658 - val_loss: 0.1149 - val_accuracy: 0.9666
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2812 - accuracy: 0.9150 - val_loss: 0.0737 - val_accuracy: 0.9772
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2122 - accuracy: 0.9365 - val_loss: 0.0452 - val_accuracy: 0.9868


0.9748299319727891

In [9]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.0988 - accuracy: 0.9748


0.974829912185669

In [0]:
def get_model(params):
  return Sequential ([
    Conv2D(filters=int(params['layer']), kernel_size=(3,3), activation='relu', input_shape=(32, 32, 1)),
    Conv2D(filters=int(params['layer']), kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=int(params['layer']), kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=2*int(params['layer']), kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=2*int(params['layer']), kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=2*int(params['layer']), kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=4*int(params['layer']), kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=4*int(params['layer']), kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=4*int(params['layer']), kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(int(params['dense']), activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(int(params['dense']), activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  X_train_explosure = exposure.equalize_hist(X_train)
  X_test_explosure = exposure.equalize_hist(X_test)
  X_train_gray = color.rgb2gray(X_train_explosure).reshape(-1, 32, 32, 1)
  X_test_gray = color.rgb2gray(X_test_explosure).reshape(-1, 32, 32, 1)
  model.fit(
      X_train_gray, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      )
  
  accuracy = model.evaluate(X_test_gray, y_test)[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [28]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.7),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.7),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.7),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
    'layer': hp.choice('layer', [16, 32, 64]),
    'dense': hp.choice('dense', [512, 1024, 2048])
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.3263 - accuracy: 0.9062

 15/138 [==>...........................]
 - ETA: 0s - loss: 0.1043 - accuracy: 0.9646

 26/138 [====>.........................]
 - ETA: 0s - loss: 0.3704 - accuracy: 0.9026

 37/138 [=======>......................]
 - ETA: 0s - loss: 0.2655 - accuracy: 0.9307

 48/138 [=========>....................]
 - ETA: 0s - loss: 0.2725 - accuracy: 0.9271

 60/138 [============>.................]
 - ETA: 0s - loss: 0.2347 - accuracy: 0.9375
